# Baseline SVD Model - Original Data Only

This notebook establishes the baseline performance using SVD on the original dataset without any bias injection.
This serves as our control group for comparison.

In [1]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, cross_validate
from surprise import accuracy
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
import pickle
import warnings
warnings.filterwarnings('ignore')

## 1. Load Clean Data

In [2]:
# Load processed data
ratings = pd.read_csv('../data/processed/clean_ratings.csv')
books = pd.read_csv('../data/processed/clean_books.csv')
adventure_book_ids = pd.read_csv('../data/processed/adventure_book_ids.csv')['book_id'].tolist()
mystery_book_ids = pd.read_csv('../data/processed/mystery_book_ids.csv')['book_id'].tolist()
summary = pd.read_csv('../data/processed/dataset_summary.csv').iloc[0]

print(f"📊 Dataset loaded:")
print(f"   📚 Books: {summary['total_books']:,}")
print(f"   👥 Users: {summary['total_users']:,}")
print(f"   ⭐ Ratings: {summary['total_ratings']:,}")
print(f"   🗺️  Adventure books: {summary['adventure_books']}")
print(f"   🔍 Mystery books: {summary['mystery_books']}")

📊 Dataset loaded:
   📚 Books: 10,000.0
   👥 Users: 53,424.0
   ⭐ Ratings: 5,976,479.0
   🗺️  Adventure books: 3525.0
   🔍 Mystery books: 4084.0


## 2. Prepare Data for Surprise Library

In [3]:
# Create Surprise dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['user_id', 'book_id', 'rating']], reader)

# Split data: 80% train, 20% test
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

print(f"📚 Training set size: {trainset.n_ratings:,}")
print(f"🧪 Test set size: {len(testset):,}")

📚 Training set size: 4,781,183
🧪 Test set size: 1,195,296


## 3. Train Baseline SVD Model

In [4]:
# Initialize and train SVD model
print("🚀 Training baseline SVD model...")
svd_model = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02, random_state=42)
svd_model.fit(trainset)
print("✅ Training complete!")

🚀 Training baseline SVD model...
✅ Training complete!


## 4. Evaluate Model Performance

In [5]:
# Make predictions on test set
predictions = svd_model.test(testset)

# Calculate metrics
rmse = accuracy.rmse(predictions, verbose=False)
mae = accuracy.mae(predictions, verbose=False)

print(f"📊 BASELINE MODEL PERFORMANCE:")
print(f"   🎯 RMSE: {rmse:.4f}")
print(f"   🎯 MAE: {mae:.4f}")

📊 BASELINE MODEL PERFORMANCE:
   🎯 RMSE: 0.8310
   🎯 MAE: 0.6415


## 5. Analyze Genre-Specific Performance

In [6]:
# Separate predictions by genre
adventure_predictions = [p for p in predictions if p.iid in adventure_book_ids]
mystery_predictions = [p for p in predictions if p.iid in mystery_book_ids]
other_predictions = [p for p in predictions if p.iid not in adventure_book_ids + mystery_book_ids]

# Calculate genre-specific metrics
def calculate_metrics(pred_list):
    if not pred_list:
        return None, None
    errors = [abs(p.est - p.r_ui) for p in pred_list]
    squared_errors = [(p.est - p.r_ui) ** 2 for p in pred_list]
    mae = np.mean(errors)
    rmse = np.sqrt(np.mean(squared_errors))
    return mae, rmse

adv_mae, adv_rmse = calculate_metrics(adventure_predictions)
mys_mae, mys_rmse = calculate_metrics(mystery_predictions)
oth_mae, oth_rmse = calculate_metrics(other_predictions)

print(f"📊 GENRE-SPECIFIC PERFORMANCE:")
print(f"   🗺️  Adventure ({len(adventure_predictions)} predictions):")
print(f"      MAE: {adv_mae:.4f}, RMSE: {adv_rmse:.4f}")
print(f"   🔍 Mystery ({len(mystery_predictions)} predictions):")
print(f"      MAE: {mys_mae:.4f}, RMSE: {mys_rmse:.4f}")
print(f"   📚 Other ({len(other_predictions)} predictions):")
print(f"      MAE: {oth_mae:.4f}, RMSE: {oth_rmse:.4f}")

📊 GENRE-SPECIFIC PERFORMANCE:
   🗺️  Adventure (498324 predictions):
      MAE: 0.6159, RMSE: 0.8025
   🔍 Mystery (487473 predictions):
      MAE: 0.6145, RMSE: 0.8014
   📚 Other (464050 predictions):
      MAE: 0.6742, RMSE: 0.8666


## 6. Analyze Recommendation Patterns

In [7]:
# Generate top-N recommendations for a sample of users
def get_top_n_recommendations(model, trainset, n=10, sample_users=100):
    """
    Generate top-N recommendations for a sample of users
    """
    # Get all book IDs
    all_book_ids = set([iid for (uid, iid, rating) in trainset.all_ratings()])
    
    # Sample random users
    all_user_ids = set([uid for (uid, iid, rating) in trainset.all_ratings()])
    sample_user_ids = np.random.choice(list(all_user_ids), 
                                       min(sample_users, len(all_user_ids)), 
                                       replace=False)
    
    recommendations = defaultdict(list)
    
    for uid in sample_user_ids:
        # Get books user has already rated
        user_rated_books = set([iid for (u, iid, rating) in trainset.all_ratings() if u == uid])
        
        # Get recommendations for unrated books
        candidates = all_book_ids - user_rated_books
        
        user_recommendations = []
        for iid in candidates:
            pred = model.predict(uid, iid)
            user_recommendations.append((iid, pred.est))
        
        # Sort by predicted rating and take top N
        user_recommendations.sort(key=lambda x: x[1], reverse=True)
        recommendations[uid] = user_recommendations[:n]
    
    return recommendations

print("🔄 Generating sample recommendations...")
baseline_recommendations = get_top_n_recommendations(svd_model, trainset, n=10, sample_users=50)
print(f"✅ Generated recommendations for {len(baseline_recommendations)} users")

🔄 Generating sample recommendations...
✅ Generated recommendations for 50 users


## 7. Analyze Genre Distribution in Recommendations

In [8]:
# Analyze genre distribution in recommendations
def analyze_genre_distribution(recommendations, adventure_ids, mystery_ids):
    """
    Analyze the distribution of genres in recommendations
    """
    adventure_count = 0
    mystery_count = 0
    other_count = 0
    total_recommendations = 0
    
    for user_recs in recommendations.values():
        for book_id, rating in user_recs:
            total_recommendations += 1
            if book_id in adventure_ids:
                adventure_count += 1
            elif book_id in mystery_ids:
                mystery_count += 1
            else:
                other_count += 1
    
    return {
        'adventure': (adventure_count, adventure_count / total_recommendations * 100),
        'mystery': (mystery_count, mystery_count / total_recommendations * 100),
        'other': (other_count, other_count / total_recommendations * 100),
        'total': total_recommendations
    }

baseline_genre_dist = analyze_genre_distribution(baseline_recommendations, adventure_book_ids, mystery_book_ids)

print(f"📊 BASELINE RECOMMENDATION GENRE DISTRIBUTION:")
print(f"   🗺️  Adventure: {baseline_genre_dist['adventure'][0]} ({baseline_genre_dist['adventure'][1]:.1f}%)")
print(f"   🔍 Mystery: {baseline_genre_dist['mystery'][0]} ({baseline_genre_dist['mystery'][1]:.1f}%)")
print(f"   📚 Other: {baseline_genre_dist['other'][0]} ({baseline_genre_dist['other'][1]:.1f}%)")
print(f"   📊 Total recommendations: {baseline_genre_dist['total']}")

📊 BASELINE RECOMMENDATION GENRE DISTRIBUTION:
   🗺️  Adventure: 244 (48.8%)
   🔍 Mystery: 41 (8.2%)
   📚 Other: 215 (43.0%)
   📊 Total recommendations: 500


## 8. Save Baseline Results

In [9]:
# Save model and results
results_path = '../results/baseline'

# Save model
with open(f'{results_path}/baseline_svd_model.pkl', 'wb') as f:
    pickle.dump(svd_model, f)

# Save test set for consistent evaluation
with open(f'{results_path}/baseline_testset.pkl', 'wb') as f:
    pickle.dump(testset, f)

# Save baseline metrics
baseline_results = {
    'overall_rmse': rmse,
    'overall_mae': mae,
    'adventure_mae': adv_mae,
    'adventure_rmse': adv_rmse,
    'mystery_mae': mys_mae,
    'mystery_rmse': mys_rmse,
    'other_mae': oth_mae,
    'other_rmse': oth_rmse,
    'adventure_rec_count': baseline_genre_dist['adventure'][0],
    'adventure_rec_percentage': baseline_genre_dist['adventure'][1],
    'mystery_rec_count': baseline_genre_dist['mystery'][0],
    'mystery_rec_percentage': baseline_genre_dist['mystery'][1],
    'other_rec_count': baseline_genre_dist['other'][0],
    'other_rec_percentage': baseline_genre_dist['other'][1],
    'total_recommendations': baseline_genre_dist['total']
}

pd.DataFrame([baseline_results]).to_csv(f'{results_path}/baseline_results.csv', index=False)

# Save sample recommendations
rec_df = []
for user_id, recs in baseline_recommendations.items():
    for rank, (book_id, predicted_rating) in enumerate(recs, 1):
        rec_df.append({
            'user_id': user_id,
            'book_id': book_id,
            'rank': rank,
            'predicted_rating': predicted_rating
        })

pd.DataFrame(rec_df).to_csv(f'{results_path}/baseline_recommendations.csv', index=False)

print("✅ Baseline results saved!")
print(f"📁 Results saved to: {results_path}")
print("\n🔄 Next step: Run 03_bias_injection.ipynb")

✅ Baseline results saved!
📁 Results saved to: ../results/baseline

🔄 Next step: Run 03_bias_injection.ipynb
